In [2]:
from mmengine.utils import get_git_hash
from mmengine.utils.dl_utils import collect_env as collect_base_env

import mmdet


def collect_env():
    """Collect the information of the running environments."""
    env_info = collect_base_env()
    env_info['MMDetection'] = f'{mmdet.__version__}+{get_git_hash()[:7]}'
    return env_info


if __name__ == '__main__':
    for name, val in collect_env().items():
        print(f'{name}: {val}')

sys.platform: linux
Python: 3.7.10 (default, Feb 26 2021, 18:47:35) [GCC 7.3.0]
CUDA available: True
MUSA available: False
numpy_random_seed: 2147483648
GPU 0: NVIDIA GeForce RTX 4090
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.1, V11.1.105
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.9.0
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2021.2-Product Build 20210312 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_8

In [3]:
import pandas as pd
import cv2
import numpy as np
import PIL.Image as Image
import os

In [4]:
import os.path as osp
import mmcv
from mmengine.fileio import dump, load
import pandas as pd

labels = ['Atelectasis',
 'Cardiomegaly',
 'Effusion',
 'Infiltrate',
 'Mass',
 'Nodule',
 'Pneumonia',
 'Pneumothorax']


In [5]:
categories = [{"id":labels.index(x), "name":x} for x in labels]
categories

[{'id': 0, 'name': 'Atelectasis'},
 {'id': 1, 'name': 'Cardiomegaly'},
 {'id': 2, 'name': 'Effusion'},
 {'id': 3, 'name': 'Infiltrate'},
 {'id': 4, 'name': 'Mass'},
 {'id': 5, 'name': 'Nodule'},
 {'id': 6, 'name': 'Pneumonia'},
 {'id': 7, 'name': 'Pneumothorax'}]

In [6]:
def xywh_to_xyxy(xywh):
    """
    Convert XYWH format (x,y center point and width, height) to XYXY format (x,y top left and x,y bottom right).
    :param xywh: [X, Y, W, H]
    :return: [X1, Y1, X2, Y2]
    """
    if np.array(xywh).ndim > 1 or len(xywh) > 4:
        raise ValueError('xywh format: [x1, y1, width, height]')
    x1 = xywh[0] - xywh[2] / 2
    y1 = xywh[1] - xywh[3] / 2
    x2 = xywh[0] + xywh[2] / 2
    y2 = xywh[1] + xywh[3] / 2
    return np.array([x1, y1, x2, y2])

In [8]:
def convert_xray_to_coco(ann_file, out_file, image_prefix):
    df = pd.read_csv(ann_file)
    
    obj_count = 0
    annotations = []
    images = []
    
    means = []
    stds = []
    
    for idx, row in df.iterrows():
        label = row["Label"]
        filename =  row["Image"]
        img_path = osp.join(image_prefix, filename)
        image = mmcv.imread(img_path)
        height, width = image.shape[:2]
        channels = cv2.split(image)
        for channel in channels:
            means.append(np.mean(channel))
            stds.append(np.std(channel))
        images.append(
            dict(id=idx, file_name=filename, height=height, width=width))
        x,y,w,h =row[["x", "y", "w", "h"]]
        
        data_anno = dict(
                image_id=idx,
                id=obj_count,
                category_id=labels.index(label),
                bbox=[x,y,w,h],
                area=row["w"] * row["h"],
                iscrowd=0)
        annotations.append(data_anno)
        obj_count += 1
    
    means = np.array(means).reshape(-1, 3)
    stds = np.array(stds).reshape(-1, 3)
    
    # Compute the overall mean and std for each channel
    mean_per_channel = np.mean(means, axis=0)
    std_per_channel = np.mean(stds, axis=0)
    
    print(mean_per_channel)
    print(std_per_channel)
    
    coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=categories)
    dump(coco_format_json, out_file)
convert_xray_to_coco("/data/train.csv", "/data/train.json", "/data/images")
convert_xray_to_coco("/data/test.csv", "/data/test.json", "/data/images")

[120.47771557 120.47771557 120.47771557]
[59.45722038 59.45722038 59.45722038]
[123.20065048 123.20065048 123.20065048]
[57.97565126 57.97565126 57.97565126]


In [ ]:
mean = [120.47771557,120.47771557,120.47771557]
std = [59.45722038,59.45722038,59.45722038]

In [ ]:
mean = [123.20065048,123.20065048,123.20065048]
std = [57.97565126,57.97565126,57.97565126]

In [17]:
!mkdir ./checkpoints

mkdir: cannot create directory ‘./checkpoints’: File exists


In [18]:

!/opt/conda/bin/mim download mmdet --config rtmdet_tiny_8xb32-300e_coco --dest ./checkpoints

processing rtmdet_tiny_8xb32-300e_coco...
rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth exists in /home/user/checkpoints
Successfully dumped rtmdet_tiny_8xb32-300e_coco.py to /home/user/checkpoints


In [22]:
config_balloon = """
# Inherit and overwrite part of the config based on this config
_base_ = '/home/user/xray/checkpoints/rtmdet_tiny_8xb32-300e_coco.py'

data_root = '/data/images/' # dataset root

train_batch_size_per_gpu = 48
train_num_workers = 16

max_epochs = 100
stage2_num_epochs = 1
base_lr = 0.00008


metainfo = {
    'classes': ('Atelectasis',
 'Cardiomegaly',
 'Effusion',
 'Infiltrate',
 'Mass',
 'Nodule',
 'Pneumonia',
 'Pneumothorax', ),
    'palette': [
        (220, 20, 60),
        (200, 201, 60),
        (184, 20, 50),
        (140, 50, 60),
        (23, 20, 60),
        (55, 20, 60),
        (77, 20, 60),
        (10, 20, 60),
        
    ]
}

train_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img=''),
        ann_file='train.json'))

val_dataloader = dict(
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img=''),
        ann_file='test.json'))

test_dataloader = val_dataloader

val_evaluator = dict(ann_file=data_root + 'test.json')

test_evaluator = val_evaluator

model = dict(bbox_head=dict(num_classes=8))

# learning rate
param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=1.0e-5,
        by_epoch=False,
        begin=0,
        end=10),
    dict(
        # use cosine lr from 10 to 20 epoch
        type='CosineAnnealingLR',
        eta_min=base_lr * 0.05,
        begin=max_epochs // 2,
        end=max_epochs,
        T_max=max_epochs // 2,
        by_epoch=True,
        convert_to_iter_based=True),
]

train_pipeline_stage2 = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='RandomResize',
        scale=(1024, 1024),
        ratio_range=(0.1, 2.0),
        keep_ratio=True),
    dict(type='RandomCrop', crop_size=(1024, 1024)),
    dict(type='YOLOXHSVRandomAug'),
    dict(type='RandomFlip', prob=0.5),
    dict(type='Pad', size=(1024, 1024), pad_val=dict(img=(114, 114, 114))),
    dict(type='PackDetInputs')
]

# optimizer
optim_wrapper = dict(
    _delete_=True,
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=base_lr, weight_decay=0.05),
    paramwise_cfg=dict(
        norm_decay_mult=0, bias_decay_mult=0, bypass_duplicate=True))

default_hooks = dict(
    checkpoint=dict(
        interval=5,
        max_keep_ckpts=2,  # only keep latest 2 checkpoints
        save_best='auto'
    ),
    logger=dict(type='LoggerHook', interval=5))

custom_hooks = [
    dict(
        type='PipelineSwitchHook',
        switch_epoch=max_epochs - stage2_num_epochs,
        switch_pipeline=train_pipeline_stage2)
]

# load COCO pre-trained weight
load_from = '/home/user/xray/checkpoints/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'

train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=max_epochs, val_interval=1)
visualizer = dict(vis_backends=[dict(type='LocalVisBackend'),dict(type='TensorboardVisBackend')])
"""

with open('/home/user/xray/configs/rtmdet_tiny_1xb4-20e_xray.py', 'w') as f:
    f.write(config_balloon)

In [ ]:
!/opt/conda/bin/python3 /mmdetection/tools/train.py /home/user/xray/configs/rtmdet_tiny_1xb4-20e_xray.py